In [174]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import numpy as np
from scipy.stats import poisson,skellam
import statsmodels.api as sm
import statsmodels.formula.api as smf

libraries

In [228]:
seriea_AVA = pd.read_csv("Group-A-v-Group-A-Cluster-Averages.csv")
seriea_AVA = seriea_AVA[['HomeTeam','AwayTeam','FTHG','FTAG']]
seriea_AVA = seriea_AVA.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})

seriea_AVB = pd.read_csv("Group-A-v-Group-B-Cluster-Averages.csv")
seriea_AVB = seriea_AVB[['HomeTeam','AwayTeam','FTHG','FTAG']]
seriea_AVB = seriea_AVB.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})

seriea_BVB = pd.read_csv("Group-B-v-Group-B-Cluster-Averages.csv")
seriea_BVB = seriea_BVB[['HomeTeam','AwayTeam','FTHG','FTAG']]
seriea_BVB = seriea_BVB.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})

In [229]:
goal_model_data_AA = pd.concat([seriea_AVA[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           seriea_AVA[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model_AA = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data_AA, 
                        family=sm.families.Poisson()).fit()

goal_model_data_AB = pd.concat([seriea_AVB[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           seriea_AVB[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model_AB = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data_AB, 
                        family=sm.families.Poisson()).fit()

goal_model_data_BB = pd.concat([seriea_BVB[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           seriea_BVB[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model_BB = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data_BB, 
                        family=sm.families.Poisson()).fit()

def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

In [230]:
Atalantascudettos=0
Bolognascudettos=0
Empoliscudettos=0
Fiorentinascudettos=0
Veronascudettos=0
Interscudettos=0
Juventusscudettos=0
Lazioscudettos=0
Milanscudettos=0
Napoliscudettos=0
Romascudettos=0
Salernitanascudettos=0
Frosinonescudettos=0
Sassuoloscudettos=0
Cagliariscudettos=0
Torinoscudettos=0
Udinesescudettos=0
Genoascudettos=0
Leccescudettos=0
Monzascudettos=0

In [232]:
for _ in range(100):
    AtalantapointsAA=0
    InterpointsAA=0
    JuventuspointsAA=0
    LaziopointsAA=0
    MilanpointsAA=0
    NapolipointsAA=0
    RomapointsAA=0

    Atalantagames=0
    Intergames=0
    Juventusgames=0
    Laziogames=0
    Milangames=0
    Napoligames=0
    Romagames=0

    SerieAteams=['Atalanta','Inter Milan','Juventus','Lazio','AC Milan','Napoli','Roma']
    SerieApoints=[AtalantapointsAA,InterpointsAA,JuventuspointsAA,
                  LaziopointsAA,MilanpointsAA,NapolipointsAA,RomapointsAA]
    SerieAgames=[Atalantagames,Intergames,Juventusgames,Laziogames,Milangames,Napoligames,Romagames]
    for i in range(0,7):
        for j in range(0,7):
            if (i!=j):
                matchvalue=random.uniform(0,1)
                matchsim = simulate_match(poisson_model_AA, SerieAteams[i], SerieAteams[j], max_goals=10)
                homewinpercent = np.sum(np.tril(matchsim, -1))
                drawpercent=np.sum(np.diag(matchsim))
                awaywinpercent=np.sum(np.triu(matchsim, 1))
                secondbound=homewinpercent+drawpercent
                if matchvalue<homewinpercent:
                    SerieApoints[i]+=3
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                elif matchvalue<secondbound:
                    SerieApoints[i]+=1
                    SerieApoints[j]+=1
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                else:
                    SerieApoints[j]+=3
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                AtalantapointsAA=SerieApoints[0]
                InterpointsAA=SerieApoints[1]
                JuventuspointsAA=SerieApoints[2]
                LaziopointsAA=SerieApoints[3]
                MilanpointsAA=SerieApoints[4]
                NapolipointsAA=SerieApoints[5]
                RomapointsAA=SerieApoints[6]

                Atalantagames=SerieAgames[0]
                Intergames=SerieAgames[1]
                Juventusgames=SerieAgames[2]
                Laziogames=SerieAgames[3]
                Milangames=SerieAgames[4]
                Napoligames=SerieAgames[5]
                Romagames=SerieAgames[6]

                SerieAstandingpointsAA=[AtalantapointsAA,
                        InterpointsAA,JuventuspointsAA,LaziopointsAA,MilanpointsAA,NapolipointsAA,RomapointsAA]
                SerieAstandingteamsAA=['Atalanta','Inter Milan','Juventus','Lazio','AC Milan',
                                     'Napoli','Roma']
                SerieAstandinggames=[Atalantagames,Intergames,Juventusgames,Laziogames,Milangames,Napoligames,Romagames]

    #print(SerieAstandingpointsAA)
    #print(SerieAstandinggames)

    BolognapointsAA=0
    EmpolipointsAA=0
    FiorentinapointsAA=0
    VeronapointsAA=0
    SalernitanapointsAA=0
    FrosinonepointsAA=0
    SassuolopointsAA=0
    CagliaripointsAA=0
    TorinopointsAA=0
    UdinesepointsAA=0
    GenoapointsAA=0
    LeccepointsAA=0
    MonzapointsAA=0

    SerieAstandingpointsAA2=[BolognapointsAA,EmpolipointsAA,FiorentinapointsAA,VeronapointsAA,SalernitanapointsAA,
                             FrosinonepointsAA,SassuolopointsAA,CagliaripointsAA,TorinopointsAA,UdinesepointsAA,
                             GenoapointsAA,LeccepointsAA,MonzapointsAA]
    SerieAstandingpointsAA_first=SerieAstandingpointsAA+SerieAstandingpointsAA2
    SerieAstandingteamsAA2=['Bologna','Empoli','Fiorentina','Verona','Salernitana','Frosinone','Sassuolo',
                          'Cagliari','Torino','Udinese','Genoa','Lecce','Monza']
    SerieAstandingteamsAA_first=SerieAstandingteamsAA+SerieAstandingteamsAA2

    #print(SerieAstandingpointsAA_first)

    AtalantapointsAB=0
    InterpointsAB=0
    JuventuspointsAB=0
    LaziopointsAB=0
    MilanpointsAB=0
    NapolipointsAB=0
    RomapointsAB=0
    BolognapointsAB=0
    EmpolipointsAB=0
    FiorentinapointsAB=0
    VeronapointsAB=0
    SalernitanapointsAB=0
    FrosinonepointsAB=0
    SassuolopointsAB=0
    CagliaripointsAB=0
    TorinopointsAB=0
    UdinesepointsAB=0
    GenoapointsAB=0
    LeccepointsAB=0
    MonzapointsAB=0

    Atalantagames=0
    Intergames=0
    Juventusgames=0
    Laziogames=0
    Milangames=0
    Napoligames=0
    Romagames=0
    Bolognagames=0
    Empoligames=0
    Fiorentinagames=0
    Veronagames=0
    Salernitanagames=0
    Frosinonegames=0
    Sassuologames=0
    Cagliarigames=0
    Torinogames=0
    Udinesegames=0
    Genoagames=0
    Leccegames=0
    Monzagames=0

    SerieAteams=['Atalanta','Inter Milan','Juventus','Lazio','AC Milan',
                'Napoli','Roma','Bologna','Empoli','Fiorentina','Verona','Salernitana','Frosinone',
                 'Sassuolo','Cagliari','Torino','Udinese','Genoa','Lecce','Monza']
    SerieApoints=[AtalantapointsAB,InterpointsAB,JuventuspointsAB,LaziopointsAB,MilanpointsAB,NapolipointsAB,RomapointsAB,
                BolognapointsAB,EmpolipointsAB,FiorentinapointsAB,VeronapointsAB,SalernitanapointsAB,FrosinonepointsAB,
                SassuolopointsAB,CagliaripointsAB,TorinopointsAB,UdinesepointsAB,GenoapointsAB,LeccepointsAB,MonzapointsAB]
    SerieAgames=[Atalantagames,Intergames,Juventusgames,Laziogames,Milangames,Napoligames,Romagames,Bolognagames,Empoligames,             
                 Fiorentinagames,Veronagames,Salernitanagames,Frosinonegames,Sassuologames,Cagliarigames,Torinogames,
                 Udinesegames,Genoagames,Leccegames,Monzagames]
    for i in range(0,20):
        for j in range(0,20):
            if ((i!=j) and ((i<7 and j>6) or (i>6 and j<7))):
                matchvalue=random.uniform(0,1)
                matchsim = simulate_match(poisson_model_AB, SerieAteams[i], SerieAteams[j], max_goals=10)
                homewinpercent = np.sum(np.tril(matchsim, -1))
                drawpercent=np.sum(np.diag(matchsim))
                awaywinpercent=np.sum(np.triu(matchsim, 1))
                secondbound=homewinpercent+drawpercent
                if matchvalue<homewinpercent:
                    SerieApoints[i]+=3
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                elif matchvalue<secondbound:
                    SerieApoints[i]+=1
                    SerieApoints[j]+=1
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                else:
                    SerieApoints[j]+=3
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                AtalantapointsAB=SerieApoints[0]
                InterpointsAB=SerieApoints[1]
                JuventuspointsAB=SerieApoints[2]
                LaziopointsAB=SerieApoints[3]
                MilanpointsAB=SerieApoints[4]
                NapolipointsAB=SerieApoints[5]
                RomapointsAB=SerieApoints[6]
                BolognapointsAB=SerieApoints[7]
                EmpolipointsAB=SerieApoints[8]
                FiorentinapointsAB=SerieApoints[9]
                VeronapointsAB=SerieApoints[10]
                SalernitanapointsAB=SerieApoints[11]
                FrosinonepointsAB=SerieApoints[12]
                SassuolopointsAB=SerieApoints[13]
                CagliaripointsAB=SerieApoints[14]
                TorinopointsAB=SerieApoints[15]
                UdinesepointsAB=SerieApoints[16]
                GenoapointsAB=SerieApoints[17]
                LeccepointsAB=SerieApoints[18]
                MonzapointsAB=SerieApoints[19]

                Atalantagames=SerieAgames[0]
                Intergames=SerieAgames[1]
                Juventusgames=SerieAgames[2]
                Laziogames=SerieAgames[3]
                Milangames=SerieAgames[4]
                Napoligames=SerieAgames[5]
                Romagames=SerieAgames[6]
                Bolognagames=SerieAgames[7]
                Empoligames=SerieAgames[8]
                Fiorentinagames=SerieAgames[9]
                Veronagames=SerieAgames[10]
                Salernitanagames=SerieAgames[11]
                Frosinonegames=SerieAgames[12]
                Sassuologames=SerieAgames[13]
                Cagliarigames=SerieAgames[14]
                Torinogames=SerieAgames[15]
                Udinesegames=SerieAgames[16]
                Genoagames=SerieAgames[17]
                Leccegames=SerieAgames[18]
                Monzagames=SerieAgames[19]

                SerieAstandingpointsAB=[AtalantapointsAB,
                        InterpointsAB,JuventuspointsAB,LaziopointsAB,MilanpointsAB,NapolipointsAB,RomapointsAB,
                        BolognapointsAB,EmpolipointsAB,FiorentinapointsAB,VeronapointsAB,SalernitanapointsAB,FrosinonepointsAB,
                        SassuolopointsAB,CagliaripointsAB,TorinopointsAB,UdinesepointsAB,GenoapointsAB,LeccepointsAB,
                        MonzapointsAB]
                SerieAstandingteamsAB=['Atalanta','Inter Milan','Juventus','Lazio','AC Milan',
                                     'Napoli','Roma','Bologna','Empoli','Fiorentina','Verona','Salernitana','Frosinone'
                                       ,'Sassuolo','Cagliari','Torino','Udinese','Genoa','Lecce','Monza']
                SerieAstandinggames=[Atalantagames,Intergames,Juventusgames,Laziogames,Milangames,Napoligames,Romagames,Bolognagames,Empoligames,             
                 Fiorentinagames,Veronagames,Salernitanagames,Frosinonegames,Sassuologames,Cagliarigames,Torinogames,
                 Udinesegames,Genoagames,Leccegames,Monzagames]

    #print(SerieAstandinggames)
    #print(SerieAstandingpointsAB)

    #SerieAstandingpointsAB_second=SerieAstandingpointsAB

    SerieAstandingpointsAB, SerieAstandingteamsAB = zip(*sorted(zip(SerieAstandingpointsAB, SerieAstandingteamsAB)))
    SerieAstandingpointsAB, SerieAstandingteamsAB = zip(*sorted(zip(SerieAstandingpointsAB, SerieAstandingteamsAB)))
    result1=list(reversed(SerieAstandingpointsAB))
    result2=list(reversed(SerieAstandingteamsAB))
    MaxValue=max(SerieAstandingpointsAB)
    #print(result1)
    #print(result2)

    BolognapointsAB=0
    EmpolipointsAB=0
    FiorentinapointsAB=0
    VeronapointsAB=0
    SalernitanapointsAB=0
    FrosinonepointsAB=0
    SassuolopointsAB=0
    CagliaripointsAB=0
    TorinopointsAB=0
    UdinesepointsAB=0
    GenoapointsAB=0
    LeccepointsAB=0
    MonzapointsAB=0

    Bolognagames=0
    Empoligames=0
    Fiorentinagames=0
    Veronagames=0
    Salernitanagames=0
    Frosinonegames=0
    Sassuologames=0
    Cagliarigames=0
    Torinogames=0
    Udinesegames=0
    Genoagames=0
    Leccegames=0
    Monzagames=0

    SerieAteams=['Bologna','Empoli','Fiorentina','Verona','Salernitana','Frosinone',
                 'Sassuolo','Cagliari','Torino','Udinese','Genoa','Lecce','Monza']
    SerieApoints=[BolognapointsAB,EmpolipointsAB,FiorentinapointsAB,VeronapointsAB,SalernitanapointsAB,FrosinonepointsAB,
                SassuolopointsAB,CagliaripointsAB,TorinopointsAB,UdinesepointsAB,GenoapointsAB,LeccepointsAB,MonzapointsAB]
    SerieAgames=[Bolognagames,Empoligames,             
                 Fiorentinagames,Veronagames,Salernitanagames,Frosinonegames,Sassuologames,Cagliarigames,Torinogames,
                 Udinesegames,Genoagames,Leccegames,Monzagames]
    for i in range(0,13):
        for j in range(0,13):
            if (i!=j):
                matchvalue=random.uniform(0,1)
                matchsim = simulate_match(poisson_model_BB, SerieAteams[i], SerieAteams[j], max_goals=10)
                homewinpercent = np.sum(np.tril(matchsim, -1))
                drawpercent=np.sum(np.diag(matchsim))
                awaywinpercent=np.sum(np.triu(matchsim, 1))
                secondbound=homewinpercent+drawpercent
                if matchvalue<homewinpercent:
                    SerieApoints[i]+=3
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                elif matchvalue<secondbound:
                    SerieApoints[i]+=1
                    SerieApoints[j]+=1
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                else:
                    SerieApoints[j]+=3
                    SerieAgames[i]+=1
                    SerieAgames[j]+=1
                BolognapointsAB=SerieApoints[0]
                EmpolipointsAB=SerieApoints[1]
                FiorentinapointsAB=SerieApoints[2]
                VeronapointsAB=SerieApoints[3]
                SalernitanapointsAB=SerieApoints[4]
                FrosinonepointsAB=SerieApoints[5]
                SassuolopointsAB=SerieApoints[6]
                CagliaripointsAB=SerieApoints[7]
                TorinopointsAB=SerieApoints[8]
                UdinesepointsAB=SerieApoints[9]
                GenoapointsAB=SerieApoints[10]
                LeccepointsAB=SerieApoints[11]
                MonzapointsAB=SerieApoints[12]

                Bolognagames=SerieAgames[0]
                Empoligames=SerieAgames[1]
                Fiorentinagames=SerieAgames[2]
                Veronagames=SerieAgames[3]
                Salernitanagames=SerieAgames[4]
                Frosinonegames=SerieAgames[5]
                Sassuologames=SerieAgames[6]
                Cagliarigames=SerieAgames[7]
                Torinogames=SerieAgames[8]
                Udinesegames=SerieAgames[9]
                Genoagames=SerieAgames[10]
                Leccegames=SerieAgames[11]
                Monzagames=SerieAgames[12]

                SerieAstandingpointsBB=[BolognapointsAB,EmpolipointsAB,FiorentinapointsAB,VeronapointsAB,SalernitanapointsAB,
                                        FrosinonepointsAB,SassuolopointsAB,CagliaripointsAB,TorinopointsAB,UdinesepointsAB,
                                        GenoapointsAB,LeccepointsAB,MonzapointsAB]
                SerieAstandingteamsBB=['Bologna','Empoli','Fiorentina','Verona','Salernitana','Frosinone'
                                       ,'Sassuolo','Cagliari','Torino','Udinese','Genoa','Lecce','Monza']
                SerieAstandinggames=[Bolognagames,Empoligames,             
                 Fiorentinagames,Veronagames,Salernitanagames,Frosinonegames,Sassuologames,Cagliarigames,Torinogames,
                 Udinesegames,Genoagames,Leccegames,Monzagames]

    #print(SerieAstandinggames)
    #print(SerieAstandingpointsBB)

    AtalantapointsAA=0
    InterpointsAA=0
    JuventuspointsAA=0
    LaziopointsAA=0
    MilanpointsAA=0
    NapolipointsAA=0
    RomapointsAA=0
    SerieAstandingpointsBB2=[AtalantapointsAA,InterpointsAA,JuventuspointsAA,LaziopointsAA,MilanpointsAA,NapolipointsAA,
                             RomapointsAA]
    SerieAstandingpointsBB_third=SerieAstandingpointsBB2+SerieAstandingpointsBB
    SerieAstandingteamsBB2=['Atalanta','Inter','Juventus','Lazio','Milan','Napoli','Roma']
    SerieAstandingteamsBB_third=SerieAstandingteamsBB2+SerieAstandingteamsBB

    SerieAstandingpoints_final = list(map(sum,zip(SerieAstandingpointsAA_first,SerieAstandingpointsAB_second,SerieAstandingpointsBB_third)))
    SerieAstandingteams_final=SerieAstandingteamsAB

    #SerieAstandingpoints_final, SerieAstandingteams_final = zip(*sorted(zip(SerieAstandingpoints_final, SerieAstandingteams_final)))
    #SerieAstandingpoints_final, SerieAstandingteams_final = zip(*sorted(zip(SerieAstandingpoints_final, SerieAstandingteams_final)))
    #result1=list(reversed(SerieAstandingpoints_final))
    #result2=list(reversed(SerieAstandingteamsAB))
    MaxValue=max(SerieAstandingpoints_final)
    #print(result1)
    #print(result2)

    if SerieAstandingpoints_final[0]==MaxValue:
        Atalantascudettos+=1
    elif SerieAstandingpoints_final[1]==MaxValue:
        Interscudettos+=1
    elif SerieAstandingpoints_final[2]==MaxValue:
        Juventusscudettos+=1
    elif SerieAstandingpoints_final[3]==MaxValue:
        Lazioscudettos+=1
    elif SerieAstandingpoints_final[4]==MaxValue:
        Milanscudettos+=1
    elif SerieAstandingpoints_final[5]==MaxValue:
        Napoliscudettos+=1
    elif SerieAstandingpoints_final[6]==MaxValue:
        Romascudettos+=1
    elif SerieAstandingpoints_final[7]==MaxValue:
        Bolognascudettos+=1
    elif SerieAstandingpoints_final[8]==MaxValue:
        Empoliscudettos+=1
    elif SerieAstandingpoints_final[9]==MaxValue:
        Fiorentinascudettos+=1
    elif SerieAstandingpoints_final[10]==MaxValue:
        Veronascudettos+=1
    elif SerieAstandingpoints_final[11]==MaxValue:
        Salernitanascudettos+=1
    elif SerieAstandingpoints_final[12]==MaxValue:
        Frosinonescudettos+=1
    elif SerieAstandingpoints_final[13]==MaxValue:
        Sassuoloscudettos+=1
    elif SerieAstandingpoints_final[14]==MaxValue:
        Cagliariscudettos+=1   
    elif SerieAstandingpoints_final[15]==MaxValue:
        Torinoscudettos+=1
    elif SerieAstandingpoints_final[16]==MaxValue:
        Udinesescudettos+=1
    elif SerieAstandingpoints_final[17]==MaxValue:
        Genoascudettos+=1
    elif SerieAstandingpoints_final[18]==MaxValue:
        Leccescudettos+=1
    elif SerieAstandingpoints_final[19]==MaxValue:
        Monzascudettos+=1
SerieAscudettos=[Atalantascudettos,Bolognascudettos,Empoliscudettos,Fiorentinascudettos,Veronascudettos,Interscudettos,
                Juventusscudettos,Lazioscudettos,Milanscudettos,Napoliscudettos,Romascudettos,Salernitanascudettos,
                Frosinonescudettos,Sassuoloscudettos,Cagliariscudettos,Torinoscudettos,Udinesescudettos,Genoascudettos,
                Leccescudettos,Monzascudettos]
SerieAstandingteams=['Atalanta','Bologna','Empoli','Fiorentina','Verona','Inter','Juventus','Lazio','Milan',
                    'Napoli','Roma','Salernitana','Frosinone','Sassuolo','Cagliari','Torino','Udinese','Genoa',
                    'Lecce','Monza']
SerieAscudettos, SerieAstandingteams = zip(*sorted(zip(SerieAscudettos, SerieAstandingteams)))
SerieAscudettos, SerieAstandingteams = zip(*sorted(zip(SerieAscudettos, SerieAstandingteams)))
result3=list(reversed(SerieAscudettos))
result4=list(reversed(SerieAstandingteams))
print(result3)
print(result4)

[76, 55, 53, 11, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Milan', 'Lazio', 'Inter', 'Atalanta', 'Napoli', 'Juventus', 'Verona', 'Udinese', 'Torino', 'Sassuolo', 'Salernitana', 'Roma', 'Monza', 'Lecce', 'Genoa', 'Frosinone', 'Fiorentina', 'Empoli', 'Cagliari', 'Bologna']


In [ ]:
SerieAstandingpointsAB=[AtalantapointsAB,
                        InterpointsAB,JuventuspointsAB,LaziopointsAB,MilanpointsAB,NapolipointsAB,RomapointsAB,
                    BolognapointsAB,EmpolipointsAB,FiorentinapointsAB,VeronapointsAB,SalernitanapointsAB,FrosinonepointsAB,
                    SassuolopointsAB,CagliaripointsAB,TorinopointsAB,UdinesepointsAB,GenoapointsAB,LeccepointsAB,
                    MonzapointsAB